In [16]:
from tensorflow import keras
import torch
from torch import nn
import torch.nn.functional as F
from torchsummary import summary
from functools import reduce
from operator import __add__

In [7]:
BATCH_SIZE = 128
MARCO_SHAPE =    (BATCH_SIZE, 21, 40, 6)
MARIANNA_SHAPE = (BATCH_SIZE, 6, 40, 22)

# Tensorflow model

In [8]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(1024,5, padding='same', activation='relu', input_shape=(6, 40, 22)))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(512,5,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(256,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(128,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(64,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(32,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(16,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(8,3,padding='same',activation='relu'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(16,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [9]:
model.summary(line_length=120)

Model: "sequential"
________________________________________________________________________________________________________________________
 Layer (type)                                         Output Shape                                    Param #           
 conv2d (Conv2D)                                      (None, 6, 40, 1024)                             564224            
                                                                                                                        
 batch_normalization (BatchNormalization)             (None, 6, 40, 1024)                             4096              
                                                                                                                        
 conv2d_1 (Conv2D)                                    (None, 6, 40, 512)                              13107712          
                                                                                                                        
 batch_norma

# PyTorch model

In [25]:
def build_padded_conv(in_ch, out_ch, kernel):
  padding = (kernel // 2 + (kernel - 2 * (kernel // 2)) - 1, kernel // 2)
  return nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=kernel, padding=padding)

class MariannaModel(nn.Module):
  def __init__(self, lead_count: int):
    super(MariannaModel, self).__init__()

    # model.add(keras.layers.Conv2D(1024,5, padding='same', activation='relu', input_shape=(6, 40, 22)))
    # model.add(keras.layers.BatchNormalization())
    self.conv1 = build_padded_conv(in_ch=lead_count, out_ch=1024, kernel=5)
    self.bn1 = nn.BatchNorm2d(1024)

    # model.add(keras.layers.Conv2D(512,5,padding='same',activation='relu'))
    # model.add(keras.layers.BatchNormalization())
    # model.add(keras.layers.MaxPooling2D())
    # model.add(keras.layers.Dropout(0.1))
    self.conv2 = build_padded_conv(in_ch=1024, out_ch=512, kernel=5)
    self.bn2 = nn.BatchNorm2d(512)
    self.pool2 = nn.MaxPool2d(kernel_size=2)
    self.drop2 = nn.Dropout(0.1)

    # model.add(keras.layers.Conv2D(256,3,padding='same',activation='relu'))
    # model.add(keras.layers.BatchNormalization())
    self.conv3 = build_padded_conv(in_ch=512, out_ch=256, kernel=3)
    self.bn3 = nn.BatchNorm2d(256)

    # model.add(keras.layers.Conv2D(128,3,padding='same',activation='relu'))
    # model.add(keras.layers.BatchNormalization())
    # model.add(keras.layers.MaxPooling2D())
    # model.add(keras.layers.Dropout(0.1))
    self.conv4 = build_padded_conv(in_ch=256, out_ch=128, kernel=3)
    self.bn4 = nn.BatchNorm2d(128)
    self.pool4 = nn.MaxPool2d(kernel_size=2)
    self.drop4 = nn.Dropout(0.1)
    
    # model.add(keras.layers.Conv2D(64,3,padding='same',activation='relu'))
    # model.add(keras.layers.BatchNormalization())
    # model.add(keras.layers.Dropout(0.1))
    self.conv5 = build_padded_conv(in_ch=128, out_ch=64, kernel=3)
    self.bn5 = nn.BatchNorm2d(64)
    self.drop5 = nn.Dropout(0.1)

    # model.add(keras.layers.Conv2D(32,3,padding='same',activation='relu'))
    # model.add(keras.layers.BatchNormalization())
    # model.add(keras.layers.Dropout(0.1))
    self.conv6 = build_padded_conv(in_ch=64, out_ch=32, kernel=3)
    self.bn6 = nn.BatchNorm2d(32)
    self.drop6 = nn.Dropout(0.1)

    # model.add(keras.layers.Conv2D(16,3,padding='same',activation='relu'))
    # model.add(keras.layers.BatchNormalization())
    # model.add(keras.layers.Dropout(0.1))
    self.conv7 = build_padded_conv(in_ch=32, out_ch=16, kernel=3)
    self.bn7 = nn.BatchNorm2d(16)
    self.drop7 = nn.Dropout(0.1)

    # model.add(keras.layers.Conv2D(8,3,padding='same',activation='relu'))
    self.conv8 = build_padded_conv(in_ch=16, out_ch=8, kernel=3)

    # model.add(keras.layers.Flatten())
    # model.add(keras.layers.Dense(64,activation='relu'))
    # model.add(keras.layers.Dense(32,activation='relu'))
    # model.add(keras.layers.Dense(16,activation='relu'))
    # model.add(keras.layers.Dense(1,activation='sigmoid'))
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(80, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 16)
    self.fc4 = nn.Linear(16, 1)

  def forward(self, x):
    x = self.bn1(F.relu(self.conv1(x)))

    x = self.bn2(F.relu(self.conv2(x)))
    x = self.drop2(self.pool2(x))

    x = self.bn3(F.relu(self.conv3(x)))

    x = self.bn4(F.relu(self.conv4(x)))
    x = self.drop4(self.pool4(x))

    x = self.bn5(F.relu(self.conv5(x)))
    x = self.drop5(x)

    x = self.bn6(F.relu(self.conv6(x)))
    x = self.drop6(x)

    x = self.bn7(F.relu(self.conv7(x)))
    x = self.drop7(x)

    x = F.relu(self.conv8(x))
    x = self.flatten(x)

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = torch.sigmoid(self.fc4(x))

    return x

In [26]:
MARIANNA_SHAPE[1:][::-1], MARCO_SHAPE[1:]

((22, 40, 6), (21, 40, 6))

In [28]:
INPUT_SHAPE = MARCO_SHAPE[1:]

lead_count = INPUT_SHAPE[0]
pytorch_model = MariannaModel(lead_count)
summary(pytorch_model, input_size=INPUT_SHAPE)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 1024, 40, 6]         538,624
       BatchNorm2d-2          [-1, 1024, 40, 6]           2,048
            Conv2d-3           [-1, 512, 40, 6]      13,107,712
       BatchNorm2d-4           [-1, 512, 40, 6]           1,024
         MaxPool2d-5           [-1, 512, 20, 3]               0
           Dropout-6           [-1, 512, 20, 3]               0
            Conv2d-7           [-1, 256, 20, 3]       1,179,904
       BatchNorm2d-8           [-1, 256, 20, 3]             512
            Conv2d-9           [-1, 128, 20, 3]         295,040
      BatchNorm2d-10           [-1, 128, 20, 3]             256
        MaxPool2d-11           [-1, 128, 10, 1]               0
          Dropout-12           [-1, 128, 10, 1]               0
           Conv2d-13            [-1, 64, 10, 1]          73,792
      BatchNorm2d-14            [-1, 64